# ML TOOL 

<span> A jupyter notebook tool to aid the testing of machine learning (ML) models in the domain of activity detection.  </span>

## 1. Set Up Jupyter NoteBook
<span style='background :yellow' > **US-01**</span> - Ensure jupyter notebook has python3.x 

In [1]:
from platform import python_version

print(python_version())

3.8.10


### 1.1 Installing Dependencies


In [2]:
%pip install torch==1.10.1 torchvision==0.11.2 torchaudio==0.10.1 -f https://download.pytorch.org/ehl/torch_stable.html
%pip install timm==0.4.12
%pip install scikit-learn
%pip install numpy

Looking in links: https://download.pytorch.org/ehl/torch_stable.html
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\rcwww\desktop\y3t1\ict3104-software-management\project\t01-nvidia-jupyternotebookenv\venv_382\scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\rcwww\desktop\y3t1\ict3104-software-management\project\t01-nvidia-jupyternotebookenv\venv_382\scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\rcwww\desktop\y3t1\ict3104-software-management\project\t01-nvidia-jupyternotebookenv\venv_382\scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\rcwww\desktop\y3t1\ict3104-software-management\project\t01-nvidia-jupyternotebookenv\venv_382\scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\rcwww\desktop\y3t1\ict3104-software-management\project\t01-nvidia-jupyternotebookenv\venv_382\scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\rcwww\desktop\y3t1\ict3104-software-management\project\t01-nvidia-jupyternotebookenv\venv_382\scripts\python.exe -m pip install --upgrade pip' command.


### Dependencies and Setup

In [1]:
import torch
import os
import ipywidgets as widgets
from subprocess import Popen
print(torch.__version__)

1.10.1+cpu


## 1. Data Exploration

Load and display video data from the Toyota Smarthome (TSU) project.

### 1.1 Datasets Folders Creation 
Initialise <span style='background :yellow' >TSU</span> Dataset in `data` folder.
- `data` folder that stores different subfolders to store different types of datasets.

In [3]:
# Installation of required Libraries 
!pip install ipywidgets
!pip install jupyter-js-widgets-nbextension
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

  Using cached https://files.pythonhosted.org/packages/e4/56/990c10ca8751182ace2464cb0e4baafb7087a40c185c9142b9cd18683fac/ipywidgets-8.0.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d8/52/2f4b8f5975312fb58f4eacab2e6f6cfd2efd05704514a60a151a4e69d608/jupyterlab_widgets-3.0.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d7/ae/ee70b20dc836d935a9a6483339854c09d8752e55a8104668e2426cf3baf3/widgetsnbextension-4.0.3-py3-none-any.whl
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/40/fd/fb94c8b6493b8f3f1daba95e8c5e92e97c95e9b72038362637e979cf747c/jupyter_js_widgets_nbextension-0.0.2.dev0-py2.py3-none-any.whl
You should consider upgrading via the 'pip install --upgrade pip' command.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


 <font color='red'>Note: Restart Jupyter Notebook if the widgets are not working.</font>

<span style='background :yellow' > **US-03**</span> - Data folder that stores different subfolders to store different types of datasets accordingly

In [ ]:
# Imports needed 
import os
import ipywidgets as widgets

# Create data folder if it does not exist 
data_directory = './data'

if not os.path.exists(data_directory):
    os.makedirs(data_directory)

In [6]:
# Get user to input the number of folders needed 
print("How many datasets would you like to upload?")
inputbounded = widgets.BoundedIntText(
    value=1,
    min=0,
    max=10,
    step=1,
    description='Datasets:',
    disabled=False
)
inputbounded

How many datasets would you like to upload?


BoundedIntText(value=1, description='Datasets:', max=10)

In [7]:
# Takes in user input folder name for datasets to be created in "./data/_____"
numdatasets = inputbounded.value
count = 0

while count < numdatasets:
    foldername = input("Enter Dataset Name: ")
    directory = './data/'+ foldername
    if not os.path.exists(directory):
        os.makedirs(directory)
        count += 1
        print("Folder " +foldername + " is created.")
        print("Please upload video data to this file path: "+ directory)
    else:
        print("Invalid Folder Name!")

Enter Dataset Name:May
Mayis created.
Please upload video data to this file path: ./data/May
Enter Dataset Name:June
Juneis created.
Please upload video data to this file path: ./data/June


In [19]:
# Select a folder from the datasets folder created
list_folder = []

for file in os.listdir(data_directory):
    d = os.path.join(data_directory, file)
    if os.path.isdir(d):
        dataset_folder = d.split("/")
        list_folder.append(dataset_folder[2])
        
        
folderbounded = widgets.Select(options=list_folder,
                        value=list_folder[0],
                        # rows=10,
                        description='Folder:',
                        disabled=False)
folderbounded

Select(description='Folder:', options=('June', 'May'), value='June')

In [27]:
folder_name = folderbounded.value

# Select a folder from the datasets folder created

mypath = data_directory+ "/" +folder_name+"/"

list_files = os.listdir(mypath)
        
        
filebounded = widgets.Select(options=list_files,
                        # rows=10,
                        description=folder_name+':',
                        disabled=False)
filebounded

Select(description='May:', options=('team01_ssh',), value='team01_ssh')

In [ ]:
from IPython.display import Video



### US-06
Loading a model
<span style='background :yellow' > **US-06**</span> - Loading a model

In [28]:
mylist = os.listdir("trained_model")
selectedModel = widgets.Dropdown(
    options=mylist,
    value = mylist[0],
    description = 'Choose Model',
    disabled=False
)
selectedModel

Dropdown(description='Choose Model', options=('PDAN_TSU_RGB', 'test1', 'test2'), value='PDAN_TSU_RGB')

In [3]:
selectedModel.value
#selectedModel = "PDAN_TSU_RGB"
selectedModel

Dropdown(description='Choose Model', options=('PDAN_TSU_RGB', 'test1', 'test2'), value='PDAN_TSU_RGB')

In [ ]:
#linux
Process=Popen("./Toyota_Smarthome/pipline/run_PDAN.sh %s" % (str(selectedModel)), shell=True)
#!./Toyota_Smarthome/pipline/run_PDAN.sh selectedModel